In [94]:
# Url that contains high-level model info is: 
#     https://innovation.cms.gov/initiatives/#views=models
# Url that contains JSON info for health care facilities where CMMI models are being tested is:
#     https://data.cms.gov/resource/x8pc-u7ta.json
# Socrata API endpoint for all hospitals registered with Medicare (acute care = a subset of this list) is:
#     https://data.medicare.gov/resource/rbry-mqwu.json

path = "/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/cmmi_src_text.json"
path2 = "/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/healthCareFacWhereCMMITested.json"
outfile = "/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/cmmiOut.csv"
outfile2 = "/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/cmmiOut_Models_withStates.csv"
outfile3 = "/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/machOutByModel.csv"


In [2]:
print path
print outfile

/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/cmmi_src_text.json
/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/cmmiOut.csv


In [3]:
import csv
import requests

In [4]:
# Didn't end up using this, but it might be helpful 

class cmmiModel:
    """Object to hold model info"""
    
    def __init__(self, kw, num, desc, url, st, mn, stage, auth, dis_sum, cat):        
        self.keywords = kw
        self.participants = num
        self.description = desc
        self.url = url
        self.states = st
        self.modelname = mn
        self.stage = stage
        self.authority = auth
        self.displaysummary = dis_sum
        self.category = cat
        
    def getModelName(self):
        return self.modelname
    
    def getInfo(self):
        temp = []
        temp.append(self.modelname)
        temp.append(self.description)
        return temp
    

In [5]:
def strSplit(mystring):
    return mystring.partition(":")[2]

strSplit("name: Christine")

' Christine'

In [6]:
import json
import pandas as pd
import re

states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}


In [7]:
# This code block imports /reads from the back-end of the original link David asked us to scrape for.
# path = "/Users/christine/Dropbox/Econometrica/CMMI_DatabaseProject/cmmi_src_text.json" 
# https://innovation.cms.gov/initiatives/#views=models 
#(go to developer tools:
# sources / innovation.cms.gov / resources / javascript / InnoModelFilter.js; I c/p the JSON data there to a .txt)


class BlankDict(dict):
        def __missing__(self, key):
            return u'Missing'
        
def checkStates(modelStates, stateDict):
    binaryOut = []

    for i in stateDict:
        binaryOut.append(0)
    
    # If an entry contains general info and doesn't refer to an actual model, there won't be any states listed
    if 'Missing' in modelStates:
        return binaryOut
    else:
        for state in modelStates:
            # print state
            dummy = state.strip()
            # print state, dummy 
            temp = sorted(stateDict.keys()).index(dummy)
            # print temp
            binaryOut[temp] = 1
        return binaryOut
    
                
with open(path) as json_data:
    cmmiData = json.load(json_data, object_hook=BlankDict)
    len(cmmiData) # should = 79

with open(outfile, "wb+") as csv_file:
    csv_writer = csv.writer(csv_file)   
    csv_writer.writerow(["Model_Name", "Category", "States", "Keywords", "Num_Participants", "Description", "url", 
                       "Stage", "Authority", "Display_Summary"])
    for i in cmmiData:    
        csv_writer.writerow([i[u'model_name'].encode('utf8'),
                            i[u'category'].encode('utf8'),
                            i[u'states'].encode('utf8'),
                            i[u'keywords'].encode('utf8'),
                            i[u'number_of_participants'].encode('utf8'),
                            i[u'description'].encode('utf8'),
                            i[u'url'].encode('utf8'),    
                            i[u'stage'].encode('utf8'),
                            i[u'authority'].encode('utf8'),
                            i[u'display_model_summary'].encode('utf8')])
 

 # This second file outputs a file in which each row = a model, and there is a column for each state; the model gets a 
# 1 for each state that includes model participant(s), and a 0 otherwise. 

with open(outfile2, "wb+") as csv_file2:
    csv_writer2 = csv.writer(csv_file2)
    stateAbbr = sorted(states.keys())    
       
    header = []
    header.append("Model_Name")
    header.append("Category")
    header.append("Stage")
    header.append("List of States")
    header.append("Number of States")
    # header.append("Checksum")
    # header.append("Num == check")
    
    for state in stateAbbr:
        header.append(state)

    csv_writer2.writerow(header)
    
    for j in cmmiData:
        getStates = str(j[u'states']).split(",")
           
        binaryStates = checkStates(getStates, states)
        
        row = []
        row.append(j[u'model_name'].encode('utf8'))
        row.append(j[u'category'].encode('utf8'))
        row.append(j[u'stage'].encode('utf8'))
        row.append(j[u'states'])
        # row.append(len(getStates))
        row.append(sum(binaryStates))
        # row.append(len(getStates) == sum(binaryStates))
        
        '''if(len(getStates)!= sum(binaryStates)):
            counter = 0
            print sorted(getStates)
            for i in sorted(getStates):
                print i, binaryStates[counter]
                counter += 1'''
    
        for s in binaryStates:
            row.append(s)
        
        csv_writer2.writerow(row)
    

In [8]:
len(states)

57

In [9]:
# Use the files we have created to subset and select models related to acute care:

models = pd.read_csv(outfile, index_col=0, parse_dates=True)
len(models) # 79

# (1) Remove rows where States == "Missing"; these appear to refer to tabs that correspond to general information
# about a set of initiatives, rather than a particular model. 

# This brings us from 79 rows to 46

m1 = models.loc[(models.States != "Missing"),]
m1.head(n=5)
len(m1) # 46

# (2) 

def matchString(mystr, row):
    mystr = mystr.lower()
    result = False
    
    # Check to see whether acute care is referenced
    match = re.search(r'\bacute care*\b',mystr)
    if match:
        row['AcuteCare'] = 1
        # print "match" # debugging
        result = True
    
    # Check to see if hospital is referenced
    match = re.search(r'\bhospitals*\b', mystr)
    if match:
        row['Hospital'] = 1
        # print "match2" # debugging 
        result = True
    
    # Check to see if providers are referenced
    match = re.search(r'\bproviders*\b', mystr)
    if match:
        row['Provider'] = 1
        # print "match3" # debugging 
        result = True 
    
    return result 

def findAcuteCareRefs(df):    
    columns = m1.columns
    index = m1.index
    m2 = pd.DataFrame(index=index, columns=columns)
    m2['AcuteCare'] = 0
    m2['Hospital'] = 0
    m2['Provider'] = 0

    # Examine keywords and description of each row; if the keywords
    for i in range(0,len(df)):
        alreadyIn = False # We only want to include a model once, regardless of whether it is a 1 for keyword ^ descr
        temp = df.iloc[i]
        temp['AcuteCare'] = 0
        temp['Hospital'] = 0
        temp['Provider'] = 0
        keyw = temp['Keywords']
        descr = temp['Description']
        
        # print i, keyw # debugging
        
        # Add the row to m2 if there is a match in either the 'Keywords' | 'Description' fields
        if matchString(keyw, temp):
            m2.iloc[i] = temp
            alreadyIn = True
        if matchString(descr, temp):
            if ~alreadyIn:
                m2.iloc[i] = temp
    # Return m2 (pd.DataFrame object) 
    return m2

# Only keep the observations whose keywords or descriptions reference acute care
m2 = findAcuteCareRefs(m1)
m2 = m2[m2.Category.notnull()]

# This is the subset of models that we should focus on, from the original 79 rows 
acuteCareModels = m2.index
acuteCareModels

Index([u'BPCI Model 1: Retrospective Acute Care Hospital Stay Only',
       u'BPCI Model 2: Retrospective Acute & Post Acute Care Episode',
       u'BPCI Model 3: Retrospective Post Acute Care Only',
       u'BPCI Model 4: Prospective Acute Care Hospital Stay Only',
       u'Community-based Care Transitions Program',
       u'Frontier Extended Stay Clinic Demonstration',
       u'Graduate Nurse Education Demonstration',
       u'Medicaid Emergency Psychiatric Demonstration',
       u'Physician Hospital Collaboration Demonstration', u'Pioneer ACO Model',
       u'Rural Community Hospital Demonstration', u'Maryland All-Payer Model',
       u'Frontier Community Health Integration Project Demonstration',
       u'Next Generation ACO Model'],
      dtype='object', name=u'Model_Name')

In [10]:
# Here, we import the names + provider numbers (and other related info) for all Medicare providers, and filter to include
# only acute care hospitals. This is the possible universe of providers that we will want to check against. 

import urllib
from operator import itemgetter
# The default limit is 1000; as of 1/4/17, there were 4807 providers listed
url = "https://data.medicare.gov/resource/rbry-mqwu.json?&$limit=10000"
response = urllib.urlopen(url)

# Medicare Acute Care Providers 
macp = json.loads(response.read(), object_hook= BlankDict)  
len(macp) # should be 4807

# Now, we can subset by only selecting Acute Care Hospitals; this brings us from n= 4807 to n = 3370
mach = []

for provider in macp:
    if provider[u'hospital_type']== "Acute Care Hospitals":
        mach.append(provider)
        
len(mach) # 3370

# QA check: make sure BPCI Model 1 NJ hospitals are included
for i in range(0, len(mach)):
    mach[i][u'hospital_name'] = mach[i][u'hospital_name'].lower()
    if mach[i][u'state'] == "NJ":
        print mach[i]['hospital_name']

mach = sorted(mach, key=itemgetter('provider_id')) # sort by provider ID

machDF = pd.DataFrame(mach)
machDF.head(n=2)

hackensack university medical center
newark beth israel medical center
palisades medical center
hunterdon medical center
st mary's general hospital
holy name medical center
clara maass medical center
univ medical center of princeton at plainsboro
cape regional medical center inc
valley hospital
cooper university hospital
morristown medical center
carepoint health-christ hospital
chilton medical center
st joseph's regional medical center
st francis medical center
virtua west jersey hospitals berlin
robert wood johnson university hospital at rahway
carepoint health - bayonne medical center
trinitas regional medical center
newton medical center
our lady of lourdes medical center
deborah heart and lung center
inspira medical center vineland
riverview medical center
robert wood johnson university hospital
raritan bay medical center perth amboy division
carepoint health-hoboken university medical center
community medical center
capital health medical center - hopewell
englewood hospital and 

,address,city,county_name,effectiveness_of_care_national_comparison,effectiveness_of_care_national_comparison_footnote,efficient_use_of_medical_imaging_national_comparison,efficient_use_of_medical_imaging_national_comparison_footnote,emergency_services,hospital_name,hospital_overall_rating,...,phone_number,provider_id,readmission_national_comparison,readmission_national_comparison_footnote,safety_of_care_national_comparison,safety_of_care_national_comparison_footnote,state,timeliness_of_care_national_comparison,timeliness_of_care_national_comparison_footnote,zip_code
0,1108 ROSS CLARK CIRCLE,DOTHAN,HOUSTON,Same as the National average,NaN,Same as the National average,NaN,True,southeast alabama medical center,3,...,3347938701,010001,Same as the National average,NaN,Above the National average,NaN,AL,Same as the National average,NaN,36301
1,2505 U S HIGHWAY 431 NORTH,BOAZ,MARSHALL,Same as the National average,NaN,Below the National average,NaN,True,marshall medical center south,3,...,2565938310,010005,Above the National average,NaN,Same as the National average,NaN,AL,Above the National average,NaN,35957


In [72]:
url = "https://data.cms.gov/resource/4aqq-tcj3.json?&$limit=10000"
response = urllib.urlopen(url)

# CMS Innovation Center Model Participants 
cmmiParticip = pd.DataFrame(json.loads(response.read(), object_hook= BlankDict))
len(cmmiParticip.index) # should be 4024

cmmiModels = cmmiParticip.name_of_initiative.unique()

columns = cmmiParticip.columns
index = cmmiParticip.index
cmmiSubset = pd.DataFrame(index=index, columns= columns)

# we only want to keep participants in models that are IN our acute care models list, or that contain BPCI (for regex reasons)
for i in range(0, len(cmmiParticip.index)):
    if cmmiParticip.iloc[i, 7] in acuteCareModels or "BPCI" in cmmiParticip.iloc[i, 7]:
        cmmiParticip.iloc[i,9] = cmmiParticip.iloc[i,9].lower()
        cmmiSubset.iloc[i] = cmmiParticip.iloc[i]

cmmiSubset = cmmiSubset[cmmiSubset.category.notnull()]
len(cmmiSubset.index) #1415


1415

In [73]:
cmmiSubset[0:3]

,category,city,facebook,location_1,location_1_address,location_1_city,location_1_state,name_of_initiative,notes,organization_name,phase_2,state,state_based,street_address,twitter,unique_id,website,youtube
1,Episode-based Payment Initiatives,Hilo,NaN,"{u'type': u'Point', u'coordinates': [-155.0922...",1190 Waianuenue Ave,Hilo,HI,BPCI Initiative: Model 2,Number of Episodes: 29 // Convening Organizati...,quantum healthcare medical associates inc.,"Congestive heart failure, Diabetes, Fractures...",HI,NaN,1190 Waianuenue Ave,NaN,3050,NaN,NaN
12,Initiatives to Accelerate the Development and ...,Circle,NaN,"{u'type': u'Point', u'coordinates': [-81.52311...",605 Sullivan Ave.,Circle,NV,Frontier Community Health Integration Project ...,NaN,mccone county health center,NaN,NV,NaN,605 Sullivan Ave.,NaN,1000,NaN,NaN
13,Primary Care Transformation,Scottsdale,https://www.facebook.com/scottsdalehealthcare,"{u'type': u'Point', u'coordinates': [-111.9260...",NaN,Scottsdale,AZ,Graduate Nurse Education Demonstration,NaN,scottsdale healthcare medical center,NaN,AZ,NaN,NaN,https://twitter.com/SHCpublicinfo,1007,http://www.shc.org/,http://www.youtube.com/user/ScottsdaleHealthca...


In [13]:
# Now, let's try to match acute care hospitals from mach to facilities in our: facWhereModelsTested dataset:
generator = (hosp[u'hospital_name'] for hosp in mach)
machSet = set(generator) # Medicare Acute Care Hospitals
len(machSet) # 3251 

cmmiSubsetSet = set(cmmiSubset[u'organization_name'])
len(cmmiSubsetSet) #1306

t1 = machSet.intersection(cmmiSubsetSet) #right now this is an empty set
len(t1) # 163

163

In [49]:
# Make a new data frame that contains all Medicare Acute Care Hospitals that have participated in @ least one CMMI model

mergeOut = pd.merge(machDF, cmmiSubset, left_on="hospital_name", right_on="organization_name", indicator=True)

counter = 0
for i in mergeOut.columns:
    print counter, i
    counter +=1

mergeOutClean = mergeOut.drop_duplicates(subset='hospital_name', keep='first')

# for i in range(0, len(mergeOutClean.index)):
    # print mergeOutClean.iloc[i,8], mergeOutClean.iloc[i,37], mergeOutClean.iloc[i,35]

len(mergeOutClean) # should be, and is, 163

0 address
1 city_x
2 county_name
3 effectiveness_of_care_national_comparison
4 effectiveness_of_care_national_comparison_footnote
5 efficient_use_of_medical_imaging_national_comparison
6 efficient_use_of_medical_imaging_national_comparison_footnote
7 emergency_services
8 hospital_name
9 hospital_overall_rating
10 hospital_overall_rating_footnote
11 hospital_ownership
12 hospital_type
13 meets_criteria_for_meaningful_use_of_ehrs
14 mortality_national_comparison
15 mortality_national_comparison_footnote
16 patient_experience_national_comparison
17 patient_experience_national_comparison_footnote
18 phone_number
19 provider_id
20 readmission_national_comparison
21 readmission_national_comparison_footnote
22 safety_of_care_national_comparison
23 safety_of_care_national_comparison_footnote
24 state_x
25 timeliness_of_care_national_comparison
26 timeliness_of_care_national_comparison_footnote
27 zip_code
28 category
29 city_y
30 facebook
31 location_1
32 location_1_address
33 location_1_city


163

In [92]:
index = mergeOutClean.index
provInfoCols = mergeOutClean.columns[(8,19, 24),]
cmmiSubset2 = cmmiSubset.name_of_initiative.unique()
cols = provInfoCols.union(cmmiSubset2)
print cols

def order(frame,var):
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame 

machByModel = pd.DataFrame(index=index, columns=cols)
machByModel = order(machByModel,['provider_id', 'hospital_name', 'state_x'])


def fillInTable(inputDf, out, modelList):
    
    modelList = sorted(modelList)
    
    for i in range(0, len(inputDf)):
        out.iloc[i, 0] = inputDf.iloc[i, 19] # provider Id
        out.iloc[i, 1] = inputDf.iloc[i, 8] # hospital name
        out.iloc[i, 2] = inputDf.iloc[i, 24] # state name
        out.iloc[i, 3:len(out)] = 0 # fill in w/0s as placeholders; these are booleans for model participation 
        
        # Here, if the initiative for a row == a particular model, give that observation a 1 for that model. 
        def checkModel(cell, models):
            result = False
            counter = -1
            counterOut = -1
            for model in models:
                counter += 1
                if cell == model:
                    result = True                 
                    counterOut = counter 
                    print cell, model, counterOut
            return [result, counterOut]
        
        temp = inputDf.iloc[i,35] #this is the name of the initiative for each observation
        
        result = checkModel(temp, modelList)
        # print "result: " + str(result) # debugging 
        
        # If there was a match, get the index and offset and make that cell a 1
        if result[0]:
            out.iloc[i, (result[1] + 3)] = 1
        
    return out

machByModelOut = fillInTable(mergeOutClean, machByModel, cmmiSubset2)

    

Index([u'BPCI Initiative: Model 1', u'BPCI Initiative: Model 2',
       u'BPCI Initiative: Model 3', u'BPCI Initiative: Model 4',
       u'Community-based Care Transitions Program',
       u'Frontier Community Health Integration Project Demonstration',
       u'Graduate Nurse Education Demonstration',
       u'Medicaid Emergency Psychiatric Demonstration',
       u'Next Generation ACO Model', u'Pioneer ACO Model', u'hospital_name',
       u'provider_id', u'state_x'],
      dtype='object')
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiative: Model 2 1
BPCI Initiative: Model 2 BPCI Initiati

In [99]:
print machByModelOut.columns

with open(outfile3, "wb+") as csv_file:
    csv_writer = csv.writer(csv_file)   
    csv_writer.writerow(machByModelOut.columns)
    for i in range(0, len(machByModelOut)): 
        csv_writer.writerow(machByModelOut.iloc[i, 0:len(machByModelOut.columns)])

        
# This is almost what we need.
# But, a few key problems:
# (1) This data is current and does not reflect or include providers that participated in past models or in models where they exited
# (for example, only Kansas is included for participation in BPCI Model 1)
# (2) Need to validate that this is the correct subset of models and that we've included all that refer to acute care providers
# (3) Element of time is still missing. Need to get start and end dates for each model and THEN assess participation by qtr-yr-prvdr
# (4)

Index([u'provider_id', u'hospital_name', u'state_x',
       u'BPCI Initiative: Model 1', u'BPCI Initiative: Model 2',
       u'BPCI Initiative: Model 3', u'BPCI Initiative: Model 4',
       u'Community-based Care Transitions Program',
       u'Frontier Community Health Integration Project Demonstration',
       u'Graduate Nurse Education Demonstration',
       u'Medicaid Emergency Psychiatric Demonstration',
       u'Next Generation ACO Model', u'Pioneer ACO Model'],
      dtype='object')
